In [1]:
!wget http://archive.ics.uci.edu/static/public/350/default+of+credit+card+clients.zip
!unzip default*

--2025-06-25 09:59:38--  http://archive.ics.uci.edu/static/public/350/default+of+credit+card+clients.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘default+of+credit+card+clients.zip’

default+of+credit+c     [   <=>              ]   5.28M  9.04MB/s    in 0.6s    

2025-06-25 09:59:39 (9.04 MB/s) - ‘default+of+credit+card+clients.zip’ saved [5539494]

Archive:  default+of+credit+card+clients.zip
 extracting: default of credit card clients.xls  


In [11]:
import numpy as np
import pandas as pd

# Load the file directly into a NumPy array
data = pd.read_excel("/content/default of credit card clients.xls", header=1)

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler

# Assuming your DataFrame is called `data`

# 1. Drop the `ID` column
data = data.drop(columns=["ID"])

# Add random noise

In [13]:
np.random.seed(32)

# Assuming 'data' is your DataFrame
num_columns = data.shape[1] - 1  # Exclude the last column

# Select 19 random column indices (excluding the last one)
random_indices = np.random.choice(range(num_columns), size=8, replace=False)

# Print the actual column indices selected
print("Selected column indices:", list(random_indices))

# Generate random noise and add it to selected columns
for idx in random_indices:
    data.iloc[:, idx] = np.random.normal(loc=0, scale=1, size=len(data))  # Adjust distribution as needed

Selected column indices: [np.int64(16), np.int64(8), np.int64(14), np.int64(20), np.int64(17), np.int64(6), np.int64(15), np.int64(0)]


/tmp/ipython-input-13-1492473599.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.72327943  1.64808753  0.08423234 ... -0.1463523   1.39976487
 -1.84141735]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[:, idx] = np.random.normal(loc=0, scale=1, size=len(data))  # Adjust distribution as needed
/tmp/ipython-input-13-1492473599.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1.84752388  0.37712259  2.548743   ... -2.84774491 -1.13579024
  0.14398742]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[:, idx] = np.random.normal(loc=0, scale=1, size=len(data))  # Adjust distribution as needed
/tmp/ipython-input-13-1492473599.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of

In [14]:
all_indices = set(range(num_columns))  # All possible column indices except the last one

non_random_indices = sorted(all_indices - set(random_indices))

# Print results
print("Selected column indices:", sorted(random_indices))
print("Non-random column indices:", non_random_indices)

Selected column indices: [np.int64(0), np.int64(6), np.int64(8), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(20)]
Non-random column indices: [1, 2, 3, 4, 5, 7, 9, 10, 11, 12, 13, 18, 19, 21, 22]


In [18]:
from sklearn.preprocessing import PowerTransformer, QuantileTransformer

X = data.drop(columns=["default payment next month"])  # All columns except target
y = data["default payment next month"]  # Target column

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.3333, random_state=42, stratify=y_temp)

pt = PowerTransformer(method="yeo-johnson")
X_train = pt.fit_transform(X_train)
X_val = pt.transform(X_val)
X_test = pt.transform(X_test)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=42)

In [19]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve

# 5. Make predictions and probabilities
y_probs = model.predict_proba(X_test_scaled)[:, 1]  # Probabilities for the positive class (Default)

# Adjust the threshold
threshold = 0.5  # Example: lower the threshold
y_pred_adjusted = (y_probs >= threshold).astype(int)

# 6. Evaluate the model with the adjusted threshold
accuracy_adjusted = accuracy_score(y_test, y_pred_adjusted)
print(f"Accuracy with adjusted threshold ({threshold}): {accuracy_adjusted:.2f}")

# Print the classification report
report_adjusted = classification_report(y_test, y_pred_adjusted, target_names=["No Default", "Default"])
print("\nClassification Report with Adjusted Threshold:\n")
print(report_adjusted)

# Optional: Analyze the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_adjusted)
print("\nConfusion Matrix with Adjusted Threshold:\n")
print(conf_matrix)

Accuracy with adjusted threshold (0.5): 0.80

Classification Report with Adjusted Threshold:

              precision    recall  f1-score   support

  No Default       0.81      0.97      0.89      5841
     Default       0.68      0.22      0.33      1659

    accuracy                           0.80      7500
   macro avg       0.75      0.59      0.61      7500
weighted avg       0.78      0.80      0.76      7500


Confusion Matrix with Adjusted Threshold:

[[5668  173]
 [1295  364]]


# Quantum model

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,MinMaxScaler , Normalizer
from torch.nn.functional import normalize
from torch.utils.data import Dataset, DataLoader , TensorDataset

In [21]:
import torch
import torch.nn as nn
import math
from functools import reduce

def get_SU_d_gens(d):
    generators = []

    # symmetric generators
    for j in range(d-1):
        for k in range(j+1, d):
            gen = torch.zeros(d, d, dtype=torch.complex128)
            gen[j, k] = 1
            gen[k, j] = 1
            generators.append(gen)

    # antisymmetric generators
    for j in range(d-1):
        for k in range(j+1, d):
            gen = torch.zeros(d, d, dtype=torch.complex128)
            gen[j, k] = -1j
            gen[k, j] = 1j
            generators.append(gen)

    # diagonal generators
    for l in range(1, d):
        gen = torch.zeros(d, d, dtype=torch.complex128)
        constant = math.sqrt(2/((l * (l+1))))
        for j in range(l):
            gen[j, j] = constant
        gen[l, l] = -constant*l
        generators.append(gen)

    return generators

class QNN(nn.Module):
    def __init__(self, qudit_dim, layers):
        torch.manual_seed(1234)
        super(QNN, self).__init__()
        self.d = qudit_dim
        self.layers = layers
        self.gens = torch.zeros(self.d**2 - 1 - 1, 1, self.d, self.d, dtype=torch.complex128)
        _gens = get_SU_d_gens(self.d)
        for i in range(self.d**2 - 1 - 1):
            self.gens[i,:,:,:] = _gens[i]
        self.qudit0 = torch.zeros(self.d, dtype=torch.complex128)
        self.qudit0[0] = 1
        self.num_gens = 23 #len(self.gens)
        self.num_params = self.num_gens * self.layers
        self.weights = nn.Parameter(torch.rand(self.num_gens, self.layers, dtype=torch.float64))

    def forward(self, x):
            weights_m = self.weights
            thetas = x.unsqueeze(1) * weights_m.t().unsqueeze(0)
            thetas = 2 * torch.atan(2*thetas)
            gensWithWeights = thetas[:, :, :, None, None] * self.gens[None, None, :, 0, :, :]
            hermitianMatrix = gensWithWeights.sum(dim=2)
            unitaryMatrix = torch.matrix_exp(1j * hermitianMatrix)
            # finalUnitary = torch.prod(unitaryMatrix, dim=1)
            finalUnitary = reduce(torch.bmm, [unitaryMatrix[:,i] for i in range(unitaryMatrix.shape[1])])
            results = torch.matmul(finalUnitary, self.qudit0)
            probalityVector = torch.abs(results)**2
            sums = torch.sum(probalityVector, dim=1).view(-1, 1)
            return probalityVector.real / sums.real


In [60]:
num_layers = 8
qnn_model = QNN(5, num_layers)


# Define the loss function and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(qnn_model.parameters(), lr=0.003)


# Define the number of epochs
num_epochs = 100

# Create DataLoader for batch processing
batch_size = 4


labels = np.eye(5)[y_train.values]


dataset_train = TensorDataset(torch.tensor(X_train_scaled) , torch.tensor(labels) )
data_loader_train = DataLoader(dataset_train,batch_size= batch_size,shuffle=True)

In [61]:
labels_test = np.eye(5)[y_test.values]

dataset_test = TensorDataset(torch.tensor(X_test_scaled) , torch.tensor(labels_test) )
data_loader_test = DataLoader(dataset_test,batch_size= batch_size,shuffle=True)

In [62]:
labels_val = np.eye(5)[y_val.values]

dataset_val = TensorDataset(torch.tensor(X_val_scaled), torch.tensor(labels_val))
data_loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

# Ridge Classification

In [63]:
# Set the L1 regularization strength (lambda)
lambda_l1 = 0.00001

training_threshold = 0.0002

prev_loss = torch.inf
no_decr = 0
epoch_threshold = 5

# Training loop
for epoch in range(num_epochs):
    qnn_model.train()
    total_loss = 0.0
    for inputs, labels in data_loader_train:

        threshold = 3.14

        optimizer.zero_grad()

        # Forward pass
        outputs = qnn_model(inputs)

        # Compute the primary loss
        labels = labels.to(float)
        loss = criterion(outputs, labels)

        # Add L1 regularization loss
        l1_loss = 0.0
        for param in qnn_model.parameters():
            l1_loss += torch.sum(torch.abs(param))  # Sum of absolute values of weights
        loss += lambda_l1 * l1_loss  # Add L1 penalty to the primary loss

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        total_loss += loss.item()

    # Print the average loss for each epoch
    average_loss = total_loss / len(data_loader_train)
    # print(f'Epoch {epoch + 1:3d}/{num_epochs}, Loss: {average_loss}')

    qnn_model.eval()
    val_loss_total = 0.0

    with torch.no_grad():
        for val_inputs, val_labels in data_loader_val:
            val_outputs = qnn_model(val_inputs)
            val_labels = val_labels.to(torch.float)
            val_loss = criterion(val_outputs, val_labels)
            val_loss_total += val_loss.item()

    average_val_loss = val_loss_total / len(data_loader_val)

    if average_val_loss + training_threshold > prev_loss:
        no_decr += 1
        if no_decr == epoch_threshold:
            qnn_model.load_state_dict(torch.load('model_weights.pth'))
            break
    else:
        torch.save(qnn_model.state_dict(), 'model_weights.pth')
        no_decr = 0
        prev_loss = average_val_loss


    print(f'Epoch {epoch + 1:3d}/{num_epochs}, '
          f'Train Loss: {average_loss:.4f}, '
          f'Val Loss: {average_val_loss:.4f} '
          f'Inc Streak: {no_decr}')



Epoch   1/100, Train Loss: 1.2534, Val Loss: 1.1616 Inc Streak: 0
Epoch   2/100, Train Loss: 1.1494, Val Loss: 1.1380 Inc Streak: 0
Epoch   3/100, Train Loss: 1.1298, Val Loss: 1.1199 Inc Streak: 0
Epoch   4/100, Train Loss: 1.1216, Val Loss: 1.1141 Inc Streak: 0
Epoch   5/100, Train Loss: 1.1191, Val Loss: 1.1143 Inc Streak: 1
Epoch   6/100, Train Loss: 1.1174, Val Loss: 1.1227 Inc Streak: 2
Epoch   7/100, Train Loss: 1.1169, Val Loss: 1.1108 Inc Streak: 0
Epoch   8/100, Train Loss: 1.1157, Val Loss: 1.1125 Inc Streak: 1
Epoch   9/100, Train Loss: 1.1146, Val Loss: 1.1154 Inc Streak: 2
Epoch  10/100, Train Loss: 1.1137, Val Loss: 1.1109 Inc Streak: 3
Epoch  11/100, Train Loss: 1.1134, Val Loss: 1.1078 Inc Streak: 0
Epoch  12/100, Train Loss: 1.1124, Val Loss: 1.1143 Inc Streak: 1
Epoch  13/100, Train Loss: 1.1114, Val Loss: 1.1100 Inc Streak: 2
Epoch  14/100, Train Loss: 1.1108, Val Loss: 1.1064 Inc Streak: 0
Epoch  15/100, Train Loss: 1.1099, Val Loss: 1.1059 Inc Streak: 0
Epoch  16/

In [64]:
qnn_model.eval() # Set the model to evaluation mode

correct_predictions = 0
total_samples =0

with torch.no_grad():
  for inputs, labels in data_loader_test:
    outputs = qnn_model(inputs)
    _, predicted = torch.max(outputs,1)
    total_samples += labels.size(0)
    correct_predictions += (predicted == labels.argmax(dim=1)).sum().item()


accuracy = correct_predictions / total_samples
print(f'Accuracy on the training set: {accuracy*100:.2f}%'   )

Accuracy on the training set: 81.59%


In [65]:
import torch

qnn_model.eval()  # Set the model to evaluation mode

all_predictions = []  # To store all predictions
all_labels = []       # To store all actual labels

with torch.no_grad():
    for inputs, labels_test in data_loader_test:
        outputs = qnn_model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the predicted class indices

        # Collect predictions and labels
        all_predictions.extend(predicted.cpu().numpy())  # Convert to numpy array and append
        all_labels.extend(labels_test.argmax(dim=1).cpu().numpy())  # Convert labels to numpy and append

In [66]:
predictions_series = pd.Series(all_predictions)
labels_series = pd.Series(all_labels)

In [67]:
predictions_series = predictions_series.clip(upper=1)

In [68]:
report = classification_report(labels_series, predictions_series)
print("\nClassification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

           0       0.83      0.96      0.89      5841
           1       0.68      0.32      0.44      1659

    accuracy                           0.82      7500
   macro avg       0.75      0.64      0.66      7500
weighted avg       0.80      0.82      0.79      7500



# Feature Importance

In [69]:
# Get the feature names (assuming X_train or X_test is a DataFrame)
feature_names = X_test.columns if isinstance(X_test, pd.DataFrame) else [f"Feature {i}" for i in range(X_test.shape[1])]

# Get the coefficients from the trained model
coefficients = model.coef_[0]  # Extract coefficients for the positive class
# coefficients = (np.arctan(coefficients) + math.pi) / 2
# Compute feature importance as the absolute value of coefficients
feature_importance = pd.DataFrame({
    "Feature": feature_names,
    "Coefficient": coefficients,
    "Importance (Abs)": np.abs(coefficients)
})

# Sort by importance (absolute value of coefficients)
feature_importance = feature_importance.sort_values(by="Importance (Abs)", ascending=False)

# Display the ranked features
print("\nFeature Importance Hierarchy:\n")
print(feature_importance)



Feature Importance Hierarchy:

       Feature  Coefficient  Importance (Abs)
5    Feature 5     4.367669          4.367669
18  Feature 18    -2.124726          2.124726
9    Feature 9     1.818317          1.818317
19  Feature 19    -1.584143          1.584143
7    Feature 7     0.734779          0.734779
21  Feature 21    -0.582316          0.582316
22  Feature 22    -0.512290          0.512290
12  Feature 12    -0.396679          0.396679
3    Feature 3    -0.377253          0.377253
11  Feature 11    -0.310975          0.310975
16  Feature 16    -0.270129          0.270129
13  Feature 13    -0.262258          0.262258
2    Feature 2    -0.243489          0.243489
20  Feature 20    -0.225211          0.225211
14  Feature 14    -0.197331          0.197331
0    Feature 0    -0.195579          0.195579
4    Feature 4     0.143747          0.143747
10  Feature 10     0.133770          0.133770
1    Feature 1    -0.115783          0.115783
6    Feature 6    -0.103076          0.103076
15

In [70]:
features_ranked = feature_importance.index.tolist()
important_model_features = features_ranked[:len(non_random_indices)]
correct_features = list(set(important_model_features) & set(non_random_indices))
print(len(correct_features),"/",len(non_random_indices))

12 / 15


In [71]:
# Prepare a dictionary to store parameter row sums
row_importance_dict = {}

# Iterate through model parameters
for name, param in qnn_model.named_parameters():
    # Compute the sum of absolute values for each row
    row_sums = param.data.abs().sum(dim=1).tolist()  # Sum absolute values across columns
    row_importance_dict[name] = row_sums

# Convert the dictionary to a Pandas DataFrame for better readability
importance_df = pd.DataFrame([
    {"Layer": name, "Row": i, "Importance (Abs Sum)": row_sum}
    for name, row_sums in row_importance_dict.items()
    for i, row_sum in enumerate(row_sums)
])

# Sort by importance
importance_df = importance_df.sort_values(by="Importance (Abs Sum)", ascending=False)

# Display the importance hierarchy
print("\nRow Importance Hierarchy:\n")
print(importance_df)


Row Importance Hierarchy:

      Layer  Row  Importance (Abs Sum)
20  weights   20             49.483876
8   weights    8             30.220924
12  weights   12             26.168966
5   weights    5              7.801484
13  weights   13              6.924599
11  weights   11              6.422790
16  weights   16              5.710393
6   weights    6              3.669586
10  weights   10              3.562099
7   weights    7              2.328237
2   weights    2              1.970546
9   weights    9              1.701363
14  weights   14              1.274420
18  weights   18              0.907506
3   weights    3              0.825735
0   weights    0              0.720018
17  weights   17              0.693309
15  weights   15              0.639069
19  weights   19              0.483899
21  weights   21              0.481304
4   weights    4              0.347537
22  weights   22              0.174884
1   weights    1              0.102948


In [72]:
features_ranked = importance_df.index.tolist()
important_model_features = features_ranked[:len(non_random_indices)]
correct_features = list(set(important_model_features) & set(non_random_indices))
print(len(correct_features),"/",len(non_random_indices))

10 / 15


In [73]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import pandas as pd
import numpy as np

# Train the Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on test set
rf_probs = rf.predict_proba(X_test)[:, 1]
rf_preds = (rf_probs >= 0.5).astype(int)

# Evaluation
print("Random Forest Metrics:")
print("Accuracy:", accuracy_score(y_test, rf_preds))
print("Precision:", precision_score(y_test, rf_preds))
print("Recall:", recall_score(y_test, rf_preds))
print("F1 Score:", f1_score(y_test, rf_preds))
print("\nClassification Report:\n", classification_report(y_test, rf_preds))

Random Forest Metrics:
Accuracy: 0.8177333333333333
Precision: 0.667816091954023
Recall: 0.350210970464135
F1 Score: 0.4594701463028865

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.95      0.89      5841
           1       0.67      0.35      0.46      1659

    accuracy                           0.82      7500
   macro avg       0.75      0.65      0.67      7500
weighted avg       0.80      0.82      0.80      7500



In [74]:
importances = rf.feature_importances_
feature_names = X_train.columns if hasattr(X_train, 'columns') else [f"feature_{i}" for i in range(X_train.shape[1])]
feature_ranking = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance Ranking:")
print(feature_ranking)


Feature Importance Ranking:
       Feature  Importance
5    feature_5    0.119664
17  feature_17    0.052909
6    feature_6    0.052537
20  feature_20    0.052171
0    feature_0    0.051796
16  feature_16    0.051759
8    feature_8    0.051527
14  feature_14    0.051352
15  feature_15    0.050754
11  feature_11    0.047500
12  feature_12    0.044466
13  feature_13    0.044456
4    feature_4    0.043328
18  feature_18    0.041943
19  feature_19    0.041858
22  feature_22    0.038114
21  feature_21    0.037682
7    feature_7    0.036329
9    feature_9    0.033782
10  feature_10    0.025566
2    feature_2    0.013636
3    feature_3    0.009401
1    feature_1    0.007469


In [75]:
features_ranked = feature_ranking.index.tolist()
important_model_features = features_ranked[:len(non_random_indices)]
correct_features = list(set(important_model_features) & set(non_random_indices))
print(len(correct_features),"/",len(non_random_indices))

7 / 15
